In [4]:
from simpleai.search import SearchProblem, astar

GOAL = 'THIS IS A LOT OF TEXT'


class HelloProblem(SearchProblem):
    def actions(self, state):
        if len(state) < len(GOAL):
            return list(' ABCDEFGHIJKLMNOPQRSTUVWXYZ')
        else:
            return []

    def result(self, state, action):
        return state + action

    def is_goal(self, state):
        return state == GOAL

    def heuristic(self, state):
        # how far are we from the goal?
        wrong = sum([1 if state[i] != GOAL[i] else 0
                    for i in range(len(state))])
        missing = len(GOAL) - len(state)
        return wrong + missing

problem = HelloProblem(initial_state='')
result = astar(problem)

print(result.state)
print(result.path())

THIS IS A LOT OF TEXT
[(None, ''), ('T', 'T'), ('H', 'TH'), ('I', 'THI'), ('S', 'THIS'), (' ', 'THIS '), ('I', 'THIS I'), ('S', 'THIS IS'), (' ', 'THIS IS '), ('A', 'THIS IS A'), (' ', 'THIS IS A '), ('L', 'THIS IS A L'), ('O', 'THIS IS A LO'), ('T', 'THIS IS A LOT'), (' ', 'THIS IS A LOT '), ('O', 'THIS IS A LOT O'), ('F', 'THIS IS A LOT OF'), (' ', 'THIS IS A LOT OF '), ('T', 'THIS IS A LOT OF T'), ('E', 'THIS IS A LOT OF TE'), ('X', 'THIS IS A LOT OF TEX'), ('T', 'THIS IS A LOT OF TEXT')]


In [2]:
from __future__ import print_function

from time import time

from copy import deepcopy

from simpleai.search import (
    backtrack, MOST_CONSTRAINED_VARIABLE, LEAST_CONSTRAINING_VALUE,
    convert_to_binary, CspProblem)

variables = ('F', 'T', 'U', 'W', 'R', 'O', 'C_10', 'C_100', 'C_1000')

domains = dict((v, list(range(1, 10))) for v in variables)


def const_different(variables, values):
    return len(values) == len(set(values))  # remove repeated values and count

constraints = [
    (('F', 'T', 'U', 'W', 'R', 'O'), const_different),
    (('O', 'R', 'C_10'), lambda vars_, values: values[0] + values[0] == values[1] + 10 * values[2]),
    (('C_10', 'W', 'U', 'C_100'), lambda vars_, values: values[0] + values[1] + values[1] == values[2] + 10 * values[3]),
    (('C_100', 'T', 'O', 'C_1000'), lambda vars_, values: values[0] + values[1] + values[1] == values[2] + 10 * values[3]),
    (('C_1000', 'F'), lambda vars_, values: values[0] == values[1])
]

original_constraints = deepcopy(constraints)
original_domains = deepcopy(domains)

start = time()
problem = CspProblem(variables, original_domains, original_constraints)
result = backtrack(problem, variable_heuristic=MOST_CONSTRAINED_VARIABLE, value_heuristic=LEAST_CONSTRAINING_VALUE)
elapsed = time() - start
print(result)
print("Took %d seconds to finish using n-ary constraints" % elapsed)


start = time()
variables, domains, constraints = convert_to_binary(variables, domains, constraints)
problem = CspProblem(variables, domains, constraints)
result = backtrack(problem, value_heuristic=LEAST_CONSTRAINING_VALUE)
elapsed = time() - start
print(result)
print("Took %d seconds to finish using binary constraints" % elapsed)

{'F': 1, 'C_1000': 1, 'T': 8, 'U': 3, 'W': 6, 'R': 4, 'O': 7, 'C_10': 1, 'C_100': 1}
Took 3 seconds to finish using n-ary constraints
{'F': 1, 'T': 8, 'U': 3, 'W': 6, 'R': 4, 'O': 7, 'C_10': 1, 'C_100': 1, 'C_1000': 1, 'hidden0': (1, 8, 3, 6, 4, 7), 'hidden1': (7, 4, 1), 'hidden2': (1, 6, 3, 1), 'hidden3': (1, 8, 7, 1)}
Took 22 seconds to finish using binary constraints


In [3]:
'''
8 puzzle problem, a smaller version of the fifteen puzzle:
http://en.wikipedia.org/wiki/Fifteen_puzzle
States are defined as string representations of the pieces on the puzzle.
Actions denote what piece will be moved to the empty space.

States must allways be inmutable. We will use strings, but internally most of
the time we will convert those strings to lists, which are easier to handle.
For example, the state (string):

'1-2-3
 4-5-6
 7-8-e'

will become (in lists):

[['1', '2', '3'],
 ['4', '5', '6'],
 ['7', '8', 'e']]

'''

from __future__ import print_function

from simpleai.search import astar, SearchProblem
from simpleai.search.viewers import WebViewer


GOAL = '''1-2-3
4-5-6
7-8-e'''

INITIAL = '''4-1-2
7-e-3
8-5-6'''


def list_to_string(list_):
    return '\n'.join(['-'.join(row) for row in list_])


def string_to_list(string_):
    return [row.split('-') for row in string_.split('\n')]


def find_location(rows, element_to_find):
    '''Find the location of a piece in the puzzle.
       Returns a tuple: row, column'''
    for ir, row in enumerate(rows):
        for ic, element in enumerate(row):
            if element == element_to_find:
                return ir, ic


# we create a cache for the goal position of each piece, so we don't have to
# recalculate them every time
goal_positions = {}
rows_goal = string_to_list(GOAL)
for number in '12345678e':
    goal_positions[number] = find_location(rows_goal, number)


class EigthPuzzleProblem(SearchProblem):
    def actions(self, state):
        '''Returns a list of the pieces we can move to the empty space.'''
        rows = string_to_list(state)
        row_e, col_e = find_location(rows, 'e')

        actions = []
        if row_e > 0:
            actions.append(rows[row_e - 1][col_e])
        if row_e < 2:
            actions.append(rows[row_e + 1][col_e])
        if col_e > 0:
            actions.append(rows[row_e][col_e - 1])
        if col_e < 2:
            actions.append(rows[row_e][col_e + 1])

        return actions

    def result(self, state, action):
        '''Return the resulting state after moving a piece to the empty space.
           (the "action" parameter contains the piece to move)
        '''
        rows = string_to_list(state)
        row_e, col_e = find_location(rows, 'e')
        row_n, col_n = find_location(rows, action)

        rows[row_e][col_e], rows[row_n][col_n] = rows[row_n][col_n], rows[row_e][col_e]

        return list_to_string(rows)

    def is_goal(self, state):
        '''Returns true if a state is the goal state.'''
        return state == GOAL

    def cost(self, state1, action, state2):
        '''Returns the cost of performing an action. No useful on this problem, i
           but needed.
        '''
        return 1

    def heuristic(self, state):
        '''Returns an *estimation* of the distance from a state to the goal.
           We are using the manhattan distance.
        '''
        rows = string_to_list(state)

        distance = 0

        for number in '12345678e':
            row_n, col_n = find_location(rows, number)
            row_n_goal, col_n_goal = goal_positions[number]

            distance += abs(row_n - row_n_goal) + abs(col_n - col_n_goal)

        return distance


result = astar(EigthPuzzleProblem(INITIAL))
# if you want to use the visual debugger, use this instead:
# result = astar(EigthPuzzleProblem(INITIAL), viewer=WebViewer())

for action, state in result.path():
    print('Move number', action)
    print(state)


Move number None
4-1-2
7-e-3
8-5-6
Move number 5
4-1-2
7-5-3
8-e-6
Move number 8
4-1-2
7-5-3
e-8-6
Move number 7
4-1-2
e-5-3
7-8-6
Move number 4
e-1-2
4-5-3
7-8-6
Move number 1
1-e-2
4-5-3
7-8-6
Move number 2
1-2-e
4-5-3
7-8-6
Move number 3
1-2-3
4-5-e
7-8-6
Move number 6
1-2-3
4-5-6
7-8-e
